In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
import json
!pip install demjson3
from demjson3 import decode
!pip install --upgrade transformers==4.2
from transformers import AutoTokenizer, AutoModel

In [ ]:
cit_dfs0 = pd.read_csv("/content/drive/MyDrive/Citation_dataframes/cit_dfs_0_finished_new_prompt_CD012768_hard.csv", index_col=0)
cit_dfs0 = cit_dfs0.astype(str)

papers = []
def extract_citation(row):
  global papers
  temp_papers = decode(row["reference_array"])
  for temp_paper in temp_papers:
    papers.append( temp_paper["citation"] )
  papers.append( row["citation"] )

useless_output = cit_dfs0.apply(extract_citation, axis=1)

In [ ]:
# crashes here: "Ran out of available RAM" ...

embeddings = []
batch = []
batch_size = 10
for i in range(0, len(papers)):
  if ( i % batch_size == 0 and i > 0 ):
    tokenizer = AutoTokenizer.from_pretrained('allenai/specter')
    model = AutoModel.from_pretrained('allenai/specter')

    inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=512)
    result = model(**inputs)
    temp_embeddings = result.last_hidden_state[:, 0, :]
    embeddings.extend(list(temp_embeddings))
    batch = []
    del tokenizer
    del model
  batch.append( papers[i] )

embeddings

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
